In [1]:
import dask.dataframe as dd
import dask_geopandas as dg
import geopandas as gpd
import pandas as pd
import utilities
from calitp import *
from shared_utils import geography_utils
from shared_utils import utils 

/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [18]:
# Original map
tmobile = dg.read_parquet("Tmobile.parquet")

In [ ]:
tmobile = utilities.breakout_districts(tmobile, utilities.GCS_FILE_PATH, 'tmobile_clipped_d9_df12', [9,10,11,12])

In [4]:
d1_4 = dg.read_parquet(
    "gs://calitp-analytics-data/data-analyses/cellular_coverage/att_clipped_d1_to_4.parquet"
)

In [5]:
d5_8 = dg.read_parquet(
    "gs://calitp-analytics-data/data-analyses/cellular_coverage/att_clipped_d5_df8.parquet"
)

In [6]:
d9_12 = dg.read_parquet(
    "gs://calitp-analytics-data/data-analyses/cellular_coverage/att_clipped_d9_df12.parquet"
)

In [7]:
att_all = [d1_4, d5_8, d9_12]

In [15]:
def concat_all_districts(all_gdf:list, gcs_file_path: str, file_name:str):
    
    # Empty dataframe
    full_gdf = pd.DataFrame()
    
    # Concat all the districts that were broken out into one
    full_gdf = dd.multi.concat(all_gdf, axis=0)
    
    # Turn it into a gdf
    full_gdf = full_gdf.compute()
    
    # Export
    utils.geoparquet_gcs_export(full_gdf, gcs_file_path,file_name)

    print('Saved to GCS')
    return full_gdf 

In [16]:
test = concat_all_districts(att_all, utilities.GCS_FILE_PATH, "att_all_counties")

Saved to GCS
